# **1. Import libraries and datasets**

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s3e16/sample_submission.csv
/kaggle/input/playground-series-s3e16/train.csv
/kaggle/input/playground-series-s3e16/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_data = pd.read_csv('/kaggle/input/playground-series-s3e16/train.csv')
train_data.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [4]:
test_data = pd.read_csv('/kaggle/input/playground-series-s3e16/test.csv')
test_data.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,74051,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,74052,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,74053,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,74054,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,74055,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


In [5]:
df = train_data.drop(['Age'], axis=1) 
df = pd.concat([df, test_data], axis=0).reset_index(drop=True)

In [6]:
train_max = train_data.shape[0] #This will help us divide the dataframe back into training and testing sets later.
train_max

74051

# **2. Basic EDA and features**

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123419 entries, 0 to 123418
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              123419 non-null  int64  
 1   Sex             123419 non-null  object 
 2   Length          123419 non-null  float64
 3   Diameter        123419 non-null  float64
 4   Height          123419 non-null  float64
 5   Weight          123419 non-null  float64
 6   Shucked Weight  123419 non-null  float64
 7   Viscera Weight  123419 non-null  float64
 8   Shell Weight    123419 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 8.5+ MB


In [8]:
train_data.corr()['Age'].sort_values(ascending=False)

Age               1.000000
Shell Weight      0.663473
Height            0.638067
Diameter          0.621256
Length            0.612843
Weight            0.601195
Viscera Weight    0.576808
Shucked Weight    0.503320
id                0.000089
Name: Age, dtype: float64

In [9]:
df.isna().sum()

id                0
Sex               0
Length            0
Diameter          0
Height            0
Weight            0
Shucked Weight    0
Viscera Weight    0
Shell Weight      0
dtype: int64

In [10]:
df.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395


In [11]:
#Change column names so that a space is not present in the name. Some models won't work with this
df.rename(columns={'Shucked Weight': 'Shucked_Weight', 'Viscera Weight': 'Viscera_Weight', 'Shell Weight': 'Shell_Weight'}, inplace=True)
test_data.rename(columns={'Shucked Weight': 'Shucked_Weight', 'Viscera Weight': 'Viscera_Weight', 'Shell Weight': 'Shell_Weight'}, inplace=True)

In [12]:
#Complete feature engineering from https://www.kaggle.com/competitions/playground-series-s3e16/discussion/415721

df['Viscera_Ratio'] = df['Viscera_Weight'] / df['Weight']
df['Shell_Ratio'] = df['Shell_Weight'] / df['Weight']
df['Shell_vs_Body'] = df['Shell_Weight'] / (df['Weight'] + df['Shell_Weight'])
df['Meat_Yield'] = df['Shucked_Weight'] / (df['Weight'] + df['Shell_Weight'])
df['Length_to_Diameter_Ratio'] = df['Length'] / df['Diameter']
df['Weight_to_VisceraWeight_Ratio'] = df['Weight'] / df['Viscera_Weight']
df['Weight_to_ShellWeight_Ratio'] = df['Weight'] / df['Shell_Weight']
df['Weight_to_ShuckedWeight_Ratio'] = df['Weight'] / df['Shucked_Weight']

df['Surface_Area'] = 2 * ((df['Length'] * df['Diameter']) + (df['Length'] * df['Height']) + (df['Diameter'] * df['Height']))
df['Volume'] = df['Length'] * df['Diameter'] * df['Height']
df['Density'] = df['Weight'] / (df['Volume'] + 0.0001)
df['Height^2'] = df['Height']**2
df['Pseudo_BMI'] = df['Weight'] / (df['Height^2'] + 0.0001)


df['Length^2'] = df['Length']**2
df['Diameter^2'] = df['Diameter']**2

df['Log_Weight'] = np.log1p(df['Weight'])

df['Weight_wo_Viscera'] = df['Shucked_Weight'] - df['Viscera_Weight']
df['Body_Condition_Index'] = np.sqrt(df['Length'] * df['Weight'] * df['Shucked_Weight'])

df['Length_Bins'] = pd.cut(df['Length'], bins=4, labels=False)

In [13]:
from sklearn.preprocessing import QuantileTransformer
num_feat = df.select_dtypes(exclude = ["object"]).columns
cat_feat = df.select_dtypes(include = ["object"]).columns
train_cat = df[cat_feat]
train_num = df[num_feat]
train_num = train_num.drop(['id'], axis=1)

QuantileTransformer(output_distribution='normal').fit_transform(train_num)

array([[ 0.63946185,  0.47100434,  0.1397103 , ...,  0.37628274,
         0.43627663,  0.08541442],
       [-0.80916817, -0.86618925, -0.80222568, ..., -0.84090635,
        -0.88719672, -1.14007693],
       [ 0.02760402,  0.14731565,  0.1397103 , ...,  0.2887499 ,
         0.12384009,  0.08541442],
       ...,
       [ 0.4403831 ,  0.40471681, -0.01380069, ..., -0.04643572,
         0.37880595,  0.08541442],
       [-0.43900121, -0.4612146 , -0.71198111, ..., -0.78162595,
        -0.55581147,  0.08541442],
       [ 1.55008498,  1.38462826,  1.01457112, ...,  0.52930109,
         1.01628666,  5.19933758]])

In [14]:
train_cat = pd.get_dummies(train_cat) #Create dummies for the categorical features
X = pd.concat([train_cat, train_num], axis=1) #Concatenate both categorical and numerical features back together.
X_test = X[train_max:].copy()
X = X[:train_max].copy() #create train and test datasets for modeling
y = train_data['Age']

# **3. Modeling**

In [15]:
import optuna
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.svm import LinearSVR
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, Normalizer, RobustScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.ensemble import (RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, 
                                GradientBoostingRegressor, HistGradientBoostingRegressor)
from sklearn.linear_model import (LinearRegression, RidgeCV, HuberRegressor, ARDRegression, Lasso, 
                                    ElasticNet, PassiveAggressiveRegressor)

seed = 42

In [16]:
scl = StandardScaler()
scl.fit(X)
scl.transform(X)
scl.transform(X_test)

array([[-0.67142688,  1.44602779, -0.75938146, ..., -0.99693561,
        -1.13668147, -1.3384645 ],
       [-0.67142688,  1.44602779, -0.75938146, ..., -0.40324347,
        -0.64635208,  0.13186089],
       [ 1.48936546, -0.69154964, -0.75938146, ..., -1.07851163,
        -0.73968641,  0.13186089],
       ...,
       [ 1.48936546, -0.69154964, -0.75938146, ..., -0.15851541,
         0.40527131,  0.13186089],
       [ 1.48936546, -0.69154964, -0.75938146, ..., -0.85644357,
        -0.68096792,  0.13186089],
       [-0.67142688, -0.69154964,  1.31686122, ...,  0.50315675,
         1.03427767,  1.60218629]])

In [17]:
#Will start by cheking different models and check their performance
models = [
    ('linear', LinearRegression()),
    ('lgb', LGBMRegressor(random_state=seed, objective='mae')),
    ('dart', LGBMRegressor(random_state=seed, boosting_type='dart')),
    ('cat', CatBoostRegressor(verbose=False)),
    ('huber', HuberRegressor(max_iter=1000000)),
    ('hgb', HistGradientBoostingRegressor(random_state=seed, loss='absolute_error')),
    ('xgb', XGBRegressor(random_state=seed, objective='reg:absoluteerror')),
]

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = 0.80, random_state=42)

In [19]:
def evaluate_model(true, predicted):
    mae = (mean_absolute_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, r2_square

In [20]:
%%time
score_list =[]
for i in range(len(models)):
    model = models[i][1]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)

    model_test_mae, model_test_r2 = evaluate_model(y_val, y_val_pred)

    
    print(f'Model performance for Test set {models[i][0]}')
    print("- Mean  Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    score_list.append([models[i][0],model_test_r2, model_test_mae])
    
    print('='*35)
    print('\n')

Model performance for Test set linear
- Mean  Absolute Error: 1.4699
- R2 Score: 0.5670


Model performance for Test set lgb
- Mean  Absolute Error: 1.3671
- R2 Score: 0.5646


Model performance for Test set dart
- Mean  Absolute Error: 1.4290
- R2 Score: 0.5268


Model performance for Test set cat
- Mean  Absolute Error: 1.4059
- R2 Score: 0.5921


Model performance for Test set huber
- Mean  Absolute Error: 1.4306
- R2 Score: 0.5514


Model performance for Test set hgb
- Mean  Absolute Error: 1.3668
- R2 Score: 0.5672


Model performance for Test set xgb
- Mean  Absolute Error: 1.3797
- R2 Score: 0.5668


CPU times: user 1min 47s, sys: 54.2 s, total: 2min 42s
Wall time: 57.7 s


## **3.2. Hyperparameter tuning with optuna**

In [21]:
# import optuna
# import lightgbm as lgb
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import train_test_split

# # Define the objective function to minimize (in this case, mean absolute error)
# def objective(trial):
#     # Define the hyperparameter search space
#     params = {
#         'objective': 'mae',
#         'metric': 'mae',
#         'boosting_type': 'gbdt',
#         'num_leaves': trial.suggest_int('num_leaves', 20, 100),
#         'max_depth': trial.suggest_int('max_depth', -1, 15),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1200),
#         'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
#         'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-10, 10),
#         'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-10, 10),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-5, 1e5),
#         'random_state': 42,
#         'n_jobs': -1
#     }
    
#     # Split the data into training and validation sets
#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
#     # Create and train the LightGBM model
#     model = lgb.LGBMRegressor(**params)
#     model.fit(X_train, y_train,
#               eval_set=[(X_val, y_val)],
#               early_stopping_rounds=50,
#               verbose=False)
    
#     # Make predictions on the validation set
#     y_pred = model.predict(X_val).round(decimals = 0)
    
#     # Calculate and return the mean absolute error
#     mae = mean_absolute_error(y_val, y_pred)
#     return mae

# # Create the Optuna study and optimize the objective function
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=200)

# # Print the best hyperparameters and the best objective value
# best_params = study.best_params
# best_mae = study.best_value
# print("Best Parameters: ", best_params)
# print("Best MAE: ", best_mae)
# lgbm = lgb.LGBMRegressor(**best_params)
# lgbm.fit(X_train, y_train)

In [22]:
# %%time
# def catboost_objective(trial):
#     params = {
#         'objective': 'MAE',
#         'loss_function': 'MAE',
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
#         'max_depth': trial.suggest_int('max_depth', 3, 12),
#         'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),
#         'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
#     }

#     classifier = CatBoostRegressor(**params,verbose = False)
#     classifier.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=20, verbose=False)

#     y_pred_proba = classifier.predict(X_val).round(decimals = 0)

#     mae = mean_absolute_error(y_val, y_pred_proba)
#     return  mae

# # Create the Optuna study and optimize the objective functions
# catboost_study = optuna.create_study(direction='minimize')
# catboost_study.optimize(catboost_objective, n_trials=100)

# # Print the best hyperparameters and corresponding ROC AUC scores - CatBoostClassifier
# catboost_best_params = catboost_study.best_params
# catboost_best_score = catboost_study.best_value
# print("CatBoost Best Hyperparameters: ", catboost_best_params)
# print("CatBoost Best ROC AUC Score: ", catboost_best_score)

# cat = CatBoostRegressor(**catboost_best_params, verbose = False)
# cat.fit(X_train, y_train)

In [23]:
# def objective(trial):
#     # Define the hyperparameter search space
#     params = {
#         'loss': 'absolute_error',
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
#         'max_iter': trial.suggest_int('max_iter', 100, 1000),
#         'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 150),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 150),
#         'max_bins': trial.suggest_int('max_bins', 2, 255),
#         'max_depth': trial.suggest_int('max_depth', 3, 15),
#         'l2_regularization': trial.suggest_loguniform('l2_regularization', 1e-10, 1e+1),
#         'random_state': 42
#     }
    
#     # Split the data into training and validation sets
#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
#     # Create and train the HGB regressor
#     model = HistGradientBoostingRegressor(**params)
#     model.fit(X_train, y_train)
    
#     # Make predictions on the validation set
#     y_pred = model.predict(X_val).round(decimals = 0)
    
#     # Calculate and return the mean absolute error
#     mae = mean_absolute_error(y_val, y_pred)
#     return mae

# # Create the Optuna study and optimize the objective function
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

# # Print the best hyperparameters and the best objective value
# best_params = study.best_params
# best_mae = study.best_value
# print("Best Parameters: ", best_params)
# print("Best MAE: ", best_mae)

# hist = HistGradientBoostingRegressor(**best_params)
# hist.fit(X_train, y_train)

In [24]:
# def objective(trial):
#     # Define the hyperparameter search space
#     params = {
#         'objective': 'reg:absoluteerror',
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
#         'max_depth': trial.suggest_int('max_depth', 3, 10),
#         'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
#         'alpha': trial.suggest_loguniform('alpha', 1e-5, 10),
#         'lambda': trial.suggest_loguniform('lambda', 1e-5, 10),
#         'random_state': 42
#     }
    
#     # Split the data into training and validation sets
#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
#     # Create and train the XGB regressor
#     model = xgb.XGBRegressor(**params)
#     model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)
    
#     # Make predictions on the validation set
#     y_pred = model.predict(X_val).round(decimals = 0)
    
#     # Calculate and return the mean absolute error
#     mae = mean_absolute_error(y_val, y_pred)
#     return mae

# # Create the Optuna study and optimize the objective function
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

# # Print the best hyperparameters and the best objective value
# best_params = study.best_params
# best_mae = study.best_value
# print("Best Parameters: ", best_params)
# print("Best MAE: ", best_mae)

# xgbr = xgb.XGBRegressor(**best_params)
# xgbr.fit(X_train, y_train)

In [25]:
# These parameters were obtained with optuna optimization
lgbm_params  = {'num_leaves': 85, 'max_depth': 5, 'learning_rate': 0.04499385904161292, 'n_estimators': 1118, 'subsample': 0.7098886961252188, 'colsample_bytree': 0.6246424547686764, 'reg_alpha': 0.6547580807681769, 'reg_lambda': 1.3956921724340055e-10, 'min_child_samples': 47, 'min_child_weight': 174.17835021793346}
cat_params = {'n_estimators': 923, 'learning_rate': 0.03172622152773958, 'max_depth': 7, 'subsample': 0.8043777871401916, 'colsample_bylevel': 0.5923058619998527}
hgb_params = {'learning_rate': 0.03826553888402566, 'max_iter': 468, 'max_leaf_nodes': 141, 'min_samples_leaf': 49, 'max_bins': 254, 'max_depth': 13, 'l2_regularization': 0.00012007615695712585}
xgbr_params = {'learning_rate': 0.35003635655666976, 'max_depth': 4, 'subsample': 0.8374263752401707, 'colsample_bytree': 0.7540439913655939, 'alpha': 8.811805887061658, 'lambda': 3.372267346667596e-05}

In [26]:
lgbm = LGBMRegressor(**lgbm_params, objective='mae')
cat = CatBoostRegressor(**cat_params, objective = 'MAE', verbose = False, loss_function = 'MAE')
hist = HistGradientBoostingRegressor(**hgb_params, loss = 'absolute_error')
xgbr = XGBRegressor(**xgbr_params, objective = 'reg:absoluteerror')

In [27]:
#Train models
lgbm.fit(X, y)
cat.fit(X, y)
hist.fit(X, y)
xgbr.fit(X, y)

XGBRegressor(alpha=8.811805887061658, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7540439913655939, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=3.372267346667596e-05,
             learning_rate=0.35003635655666976, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None, ...)

In [28]:
from sklearn.ensemble import VotingRegressor

voting_clf = VotingRegressor(estimators = [('lgbm', lgbm),
                                           ('cat', cat),
                                           ('hist', hist),
                                           ('xgbr', xgbr),  
                                           ])
voting_clf.fit(X, y)

VotingRegressor(estimators=[('lgbm',
                             LGBMRegressor(colsample_bytree=0.6246424547686764,
                                           learning_rate=0.04499385904161292,
                                           max_depth=5, min_child_samples=47,
                                           min_child_weight=174.17835021793346,
                                           n_estimators=1118, num_leaves=85,
                                           objective='mae',
                                           reg_alpha=0.6547580807681769,
                                           reg_lambda=1.3956921724340055e-10,
                                           subsample=0.7098886961252188)),
                            ('cat',
                             <catboost.co...
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          lambda=3.372267346667596e-05,
                                          learning_rate=0.35003635655666976,
                                          max_bin=None, max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=4,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs=None,
                                          num_parallel_tree=None, ...))])

In [29]:
predictions = voting_clf.predict(X_test).round(decimals = 0) #rounding the results achieves better scores.

In [30]:
output = pd.DataFrame({'id': test_data['id'], 'Age': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
